In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
data_path = r"C:\Users\yoon0\OneDrive\바탕 화면\mimic\aki_df.csv"
aki_df = pd.read_csv(data_path)
data_path = r"C:\Users\yoon0\OneDrive\바탕 화면\mimic\non_aki_df.csv"
non_aki_df = pd.read_csv(data_path)

In [ ]:
import pandas as pd

df = pd.concat([aki_df, non_aki_df], ignore_index=True)

# 이 예제에서는 이미 합쳐진 DataFrame이 `df`라고 가정합니다

# Step 2: 결측치가 3개 이상인 행 삭제
# 결측치가 3개 미만인 행만 유지
df_cleaned = df[df.isnull().sum(axis=1) < 6]

# Step 3: 특정 피처(열) 삭제
columns_to_drop = [
    'GU Irrigant/Urine Volume Out_227489',
    'Temperature Celsius_223762',
    '220734'
]
df_cleaned = df_cleaned.drop(columns=columns_to_drop)

# Step 4: `subject_id`와 `hadm_id`로 그룹화한 후 각 그룹에서 `AKI_stage`가 2 이상인 첫 번째 행 이후의 행 삭제

# `charttime`이 datetime 형식인지 확인
df_cleaned['charttime'] = pd.to_datetime(df_cleaned['charttime'])

# DataFrame을 `subject_id`, `hadm_id`, `charttime` 기준으로 정렬하여 그룹 내 시간 순서 보장
df_cleaned = df_cleaned.sort_values(['subject_id', 'hadm_id', 'charttime'])
df_final = df_cleaned



In [ ]:

# 2. 결측값 처리
# 'gender'를 제외한 모든 수치형 컬럼 식별
exclude_cols = ['subject_id', 'hadm_id', 'charttime', 'admission_date', 'gender']
numeric_cols = df_final.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

# 'gender'를 범주형 변수로 설정
categorical_cols = ['gender']

# 수치형 변수 결측값 중앙값으로 채우기
for col in numeric_cols:
    median = df_final[col].median()
    df_final[col].fillna(median, inplace=True)

# 범주형 변수 결측값 최빈값으로 채우기
for col in categorical_cols:
    mode = df_final[col].mode()[0]
    df_final[col].fillna(mode, inplace=True)

In [ ]:
#원-핫 인코딩 사용 (범주형 변수의 고유값 수가 적을 경우)
df_final = pd.get_dummies(df_final, columns=categorical_cols, drop_first=True)


In [ ]:
#scaler = MinMaxScaler()
#df_final[numeric_cols] = scaler.fit_transform(df_final[numeric_cols])


In [ ]:
#df_final.rename(columns={'gender_M': 'gender'}, inplace=True)


In [ ]:
# 5. 시간 기반 윈도우 설정 및 레이블링
df_final['charttime'] = pd.to_datetime(df_final['charttime'])
data = df_final.sort_values(['subject_id', 'hadm_id', 'charttime'])

# 윈도우 설정
window_size = pd.Timedelta(hours=24)      # 24시간 윈도우
step_size = pd.Timedelta(hours=6)        # 6시간 간격 (중첩)
future_window = pd.Timedelta(hours=48)    # 48시간 미래 윈도우

window_features = []
window_labels = []
window_groups = []

# 그룹 번호 부여
group_keys = list(data.groupby(['subject_id', 'hadm_id']).groups.keys())
group_numbers = {name: i for i, name in enumerate(group_keys)}

# 각 그룹별로 처리
for (subject, hadm), group in data.groupby(['subject_id', 'hadm_id']):
    group = group.reset_index(drop=True)
    start_time = group['charttime'].min()
    end_time = group['charttime'].max()
    
    # 마지막 가능한 윈도우 시작 시간을 계산
    last_window_start_time = end_time - future_window - window_size
    if last_window_start_time < start_time:
        # 윈도우를 설정할 수 없는 경우 (데이터가 부족한 경우)
        continue
    
    current_start = start_time
    group_num = group_numbers[(subject, hadm)]
    while current_start <= last_window_start_time:
        current_end = current_start + window_size
        
        # 현재 윈도우 내 데이터
        window_data = group[(group['charttime'] >= current_start) & (group['charttime'] < current_end)]
        
        if window_data.empty:
            current_start += step_size
            continue
        
        # 윈도우 내 수치형 피처의 평균값 계산 (피처별 평균)
        feature_vector = window_data[numeric_cols].mean().tolist()
        
        # 'gender' 값 추가 (각 그룹 내 고정)
        gender_value = group['gender_M'].iloc[0]
        feature_vector.append(gender_value)
        
        # 레이블링: 윈도우 종료 후 48시간 내 AKI_stage >=2 여부
        future_start = current_end
        future_end = current_end + future_window
        future_data = group[(group['charttime'] >= future_start) & (group['charttime'] < future_end)]
        label = 1 if (future_data['AKI_stage'] >= 2).any() else 0
        
        # 저장
        window_features.append(feature_vector)
        window_labels.append(label)
        window_groups.append(group_num)
        
        # 윈도우 이동
        current_start += step_size

# 피처 컬럼 이름 설정
feature_names = numeric_cols + ['gender']

# 데이터프레임 생성
X = pd.DataFrame(window_features, columns=feature_names)
X['group'] = window_groups
y = pd.Series(window_labels, name='aki')

# 결측값 확인 및 처리 (만약 평균 내기 때문에 결측값이 있을 수 있음)
X.fillna(0, inplace=True)  # 또는 다른 적절한 방법으로 처리


In [ ]:
print("\nX 데이터 예시:")
print(X.head())

In [ ]:
print(y.value_counts())


In [ ]:
# 결측값 확인 및 처리 (만약 평균 내기 때문에 결측값이 있을 수 있음)
X.fillna(0, inplace=True)  # 또는 다른 적절한 방법으로 처리

# 6. 데이터 분할 (훈련 세트와 테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 7. XGBoost 모델 구축 및 학습
model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train, y_train)

# 8. 예측 및 평가
y_pred = model.predict(X_test)

print("정확도 (Accuracy):", accuracy_score(y_test, y_pred))
print("분류 보고서 (Classification Report):")
print(classification_report(y_test, y_pred))